In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from fycharts.SpotifyCharts import SpotifyCharts

import numpy as np
import pandas as pd
import datetime

import warnings

warnings.filterwarnings(action='ignore')

# Crawling Top 200 chart between 2020/1/31 - 2020/6/17

In [2]:
def make_continuous_day() :
    start = datetime.date(2020,2,1)
    end = datetime.date(2020,4,30)
    all_day = end-start

    day_list = []

    for i in range(all_day.days +1) :
        delta = start + datetime.timedelta(days=i)
        day_list.append(delta)

    day = []
    for ls in day_list :
        num_date = str(ls.year) + '/' + str(ls.month) + '/' + str(ls.day)
        day.append(datetime.datetime.strptime(num_date, "%Y/%m/%d").strftime("%Y-%m-%d"))
        
    return day

In [3]:
day = make_continuous_day()

api = SpotifyCharts()
#api.top200Weekly(start = '2020-05-29', output_file='top200/5_29.csv')

# Data load

**5월 마지막주차 (2020/05/29~2020/06/05) 데이터는 에러로 제외**

In [4]:
Jan_31 = pd.read_csv('data/top200/1_31.csv', encoding= 'unicode_escape')
Feb_07 = pd.read_csv('data/top200/2_07.csv', encoding= 'unicode_escape')
Feb_14 = pd.read_csv('data/top200/2_14.csv', encoding= 'unicode_escape')
Feb_21 = pd.read_csv('data/top200/2_21.csv', encoding= 'unicode_escape')
Feb_28 = pd.read_csv('data/top200/2_28.csv', encoding= 'unicode_escape')
Mar_06 = pd.read_csv('data/top200/3_06.csv', encoding= 'unicode_escape')
Mar_13 = pd.read_csv('data/top200/3_13.csv', encoding= 'unicode_escape')
Mar_20 = pd.read_csv('data/top200/3_20.csv', encoding= 'unicode_escape')
Mar_27 = pd.read_csv('data/top200/3_27.csv', encoding= 'unicode_escape')
Api_03 = pd.read_csv('data/top200/4_03.csv', encoding= 'unicode_escape')
Api_10 = pd.read_csv('data/top200/4_10.csv', encoding= 'unicode_escape')
Api_17 = pd.read_csv('data/top200/4_17.csv', encoding= 'unicode_escape')
Api_24 = pd.read_csv('data/top200/4_24.csv', encoding= 'unicode_escape')
May_01 = pd.read_csv('data/top200/5_01.csv', encoding= 'unicode_escape')
May_08 = pd.read_csv('data/top200/5_08.csv', encoding= 'unicode_escape')
May_15 = pd.read_csv('data/top200/5_15.csv', encoding= 'unicode_escape')
May_22 = pd.read_csv('data/top200/5_22.csv', encoding= 'unicode_escape')
Jun_05 = pd.read_csv('data/top200/6_05.csv', encoding= 'unicode_escape')

**Jan_31은 nan column이 200개 생성되어 모두 drop 해줌**

In [5]:
Jan_31.dropna(axis=0, inplace=True)
Jan_31.reset_index(drop=True, inplace=True)

# Extract feature values from charts

In [6]:
def extract_feature(data) : 
    s_id = data.spotify_id

    # client information
    client_id = '5a483a6380094c8bad8def8214cccb69'
    client_secret = '75c31c725ebc4644907168e8a61cb1a5'

    #spotify method to access API
    credential = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=credential)

    #extract feature
    feature_1 = sp.audio_features(s_id[0:100])
    feature_1 = pd.DataFrame(feature_1)
    
    feature_2 = sp.audio_features(s_id[100:200])
    feature_2 = pd.DataFrame(feature_2)

    feature = pd.concat([feature_1, feature_2], axis=0)
    feature.reset_index(drop=True, inplace=True)
    
    data = pd.concat([data, feature], axis=1)
    
    return data

In [7]:
Jan_31 = extract_feature(Jan_31)
Feb_07 = extract_feature(Feb_07)
Feb_14 = extract_feature(Feb_14)
Feb_21 = extract_feature(Feb_21)
Feb_28 = extract_feature(Feb_28)
Mar_06 = extract_feature(Mar_06)
Mar_13 = extract_feature(Mar_13)
Mar_20 = extract_feature(Mar_20)
Mar_27 = extract_feature(Mar_27)
Api_03 = extract_feature(Api_03)
Api_10 = extract_feature(Api_10)
Api_17 = extract_feature(Api_17)
Api_24 = extract_feature(Api_24)
May_01 = extract_feature(May_01)
May_08 = extract_feature(May_08)
May_15 = extract_feature(May_15)
May_22 = extract_feature(May_22)
Jun_05 = extract_feature(Jun_05)

# Combine all data as one data 

In [8]:
data_list = [Jan_31, Feb_07, Feb_14, Feb_21, Feb_28, Mar_06, Mar_13, Mar_20, Mar_27, Api_03, Api_10, Api_17, Api_24, May_01, May_08, 
             May_15, May_22, Jun_05]

In [9]:
data = pd.concat(data_list, axis=0)
data.reset_index(drop=True, inplace = True)

# replace date as start date and end date

In [10]:
start = [0]*data.shape[0]
end = [0]*data.shape[0]

data['start_date'] = start
data['end_date'] = end

for row in range(data.shape[0]) :
    data['start_date'][row], data['end_date'][row] = data['date'][row].split('--')
    
data.drop(['date'], axis=1, inplace=True)

In [11]:
data.to_csv('data/data.csv')